In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import datetime

In [24]:
mu_masse_1 = 5.944892676091842
s_masse_1 = 1.0452954216771342
mu_masse_2 = 4.012421689011621
s_masse_2 = 1.2811831375376432
mu_speed_1 = 8.788235294117646
s_speed_1 = 1.9745088303442118
mu_speed_2 = 37.790625
s_speed_2 = 5.31080027956004
alpha_t_delta_1 = 0.7720943018929209
beta_t_delta_1 = 32.180388904581854
alpha_t_delta_2 = 0.6924339239082432
beta_t_delta_2 = 85.33142881264439

In [25]:
def random_masse_1():
    masse_1 = np.random.lognormal(mu_masse_1, s_masse_1, 1).round(1)
    return masse_1

def random_masse_2():
    masse_2 = np.random.lognormal(mu_masse_2, s_masse_2, 1).round(1)
    return masse_2

def random_speed_1():
    speed_1 = np.random.normal(mu_speed_1, s_speed_1, 1).round(2)
    return speed_1

def random_speed_2():
    speed_2 = np.random.normal(mu_speed_2, s_speed_2, 1).round(2)
    return speed_2

def random_t_delta_1():
    t_delta_1 = int(np.random.gamma(alpha_t_delta_1, beta_t_delta_1, 1).round(0))
    return t_delta_1

def random_t_delta_2():
    t_delta_2 = int(np.random.gamma(alpha_t_delta_2, beta_t_delta_2, 1).round(0))
    return t_delta_1

In [26]:
results = []
double_entries = []



amount_of_hours = 200

for i in range (0, amount_of_hours, 1):
    results.append(i)

entries = 5
for j in range (0, entries, 1):
    double_entries.append(j)
    
results = pd.DataFrame(results, columns= ['Hours'])
double_entries = pd.DataFrame(double_entries, columns= ['Hours'])

header = ['Masse 1', 'Speed 1', 'Masse 2', 'Speed 2','Day', 'Energie in kJ 1', 'Energie in kJ 2']

for value in header:
    results[value] = np.nan
    
for value in header: 
    double_entries[value] = np.nan
    

In [27]:
for value in header:
    results[value] = np.nan

In [28]:
end = amount_of_hours
start = 0
i = 0
j = 0
event_1 = 0
event_2 = 0

In [29]:
while i < end:
    t_delta_1 = random_t_delta_1()
    event_1 += t_delta_1
    
    masse_1 = random_masse_1()
    results['Masse 1'][event_1] = masse_1
    
    speed_1 = random_speed_1()
    results['Speed 1'][event_1] = speed_1
    
    print(event_1)
    
    i += event_1
    
while j < end:
    t_delta_2 = random_t_delta_2()
    event_2 += t_delta_2
    
    masse_2 = random_masse_2()
    results['Masse 2'][event_2] = masse_2
    
    speed_2 = random_speed_2()
    results['Speed 2'][event_2] = speed_2
    
    print(event_2)
    
    j += event_2

14
18
28
33
65
78
13
26
39
52
65
78


c:\users\m0268176\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\m0268176\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
c:\users\m0268176\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [30]:
results['Energie in kJ 1'] = results.apply(
    (lambda x: results['Masse 1'] * results['Speed 1'] * results['Speed 1'] / 2 / 1000).round(2)
)

results['Day'] = results.apply(
    lambda x: results['Hours'] % 24
)
results.fillna(0)

AttributeError: 'function' object has no attribute 'round'

In [ ]:
pd.set_option('display.max_rows', results.shape[0]+1)
results